##### CS688 Assignment 4
##### @author: Sahil Khanna
##### BU ID: U86445364
##### Created on: Nov 6, 2021

In [1]:
import requests
import bs4
import pandas as pd
import re
from splinter import Browser
import warnings
warnings.filterwarnings(action = 'ignore')

## Part 1
Collect some sample data (at least 20) from the following resources, which include
the keyword “Covid-19” and “Booster shots”.

Resources: Google-news

In [2]:
#Using chrom driver 
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

keywords = 'COVID-19 and Booster shots'.lower()  
url = f'https://news.google.com/search?q={keywords}'

browser.visit(url)
html = browser.html
#html

In [4]:
#Creat a Beautiful Soup object
soup = bs4.BeautifulSoup(html, 'html.parser')

In [21]:
articles = [] 
for i in range(0,100): 
    a = soup.select('.ipQwMb')[i].getText()
    articles.append(a)
    
for i in range(0,len(articles)):
    print("\n",articles[i])


 What to Know About COVID-19 Vaccine Boosters: An Op-Ed by NYC's Top Doctor

 How to choose a COVID-19 vaccine booster shot

 COVID-19 booster shots coming following federal approval

 Confused about which COVID booster to get? This guide will help.

 At least 89% of vaccinated American adults will qualify for Covid-19 booster shots

 NBA recommending COVID-19 booster shots to players, coaches, referees

 NBA recommends COVID-19 booster shots for players, coaches, referees | TheHill

 NBA urges players to receive COVID-19 booster shots

 NBA recommends COVID booster shots for players, coaches and referees

 NBA issues league wide mandate requiring covid-19 booster shots

 Far fewer Californians getting COVID-19 booster shots than expected, bringing new alarms

 Pfizer-BioNTech expected to seek authorization for coronavirus booster for people 18 and older

 Indonesia to start COVID-19 boosters after 50% of public vaccinated

 No, COVID-19 vaccines or boosters will not cause you to test

In [22]:
#df = pd.DataFrame({"articles": articles})
df = pd.DataFrame({"articles": articles[0:6] + articles[10:]}) #Remove duplicate NBA articles

In [23]:
df

,articles
0,What to Know About COVID-19 Vaccine Boosters: ...
1,How to choose a COVID-19 vaccine booster shot
2,COVID-19 booster shots coming following federa...
3,Confused about which COVID booster to get? Thi...
4,At least 89% of vaccinated American adults wil...
...,...
91,Nearly 1 million Covid booster shots have alre...
92,COVID-19 booster shots are coming; here's who ...
93,Johnson & Johnson Says Its Booster Shot Provid...
94,Moderna Has Asked The FDA To Authorize A Boost...


## Part 2

## Preprocessing the articles

Removing stopwords, punctuations 

In [24]:
import nltk
#nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem import LancasterStemmer, WordNetLemmatizer

In [25]:
stop_words = set(stopwords.words('english'))

lemmer = WordNetLemmatizer()

def prep(article):
    
    article = article.lower()
    
    # Remove punctuation
    article = re.sub(r'[^\w\s]', '', article)
    
    # Remove numbers
    article = re.sub(r'\d+', '', article)
    
    
    # Remove stopwords and lemmatize 
    article = [lemmer.lemmatize(w) for w in article.split() if w not in stop_words]
    return ' '.join(article) 

df['clean_articles'] = df['articles'].apply(prep)
df.head()

,articles,clean_articles
0,What to Know About COVID-19 Vaccine Boosters: ...,know covid vaccine booster oped nycs top doctor
1,How to choose a COVID-19 vaccine booster shot,choose covid vaccine booster shot
2,COVID-19 booster shots coming following federa...,covid booster shot coming following federal ap...
3,Confused about which COVID booster to get? Thi...,confused covid booster get guide help
4,At least 89% of vaccinated American adults wil...,least vaccinated american adult qualify covid ...


# Afinn Sentiment Analyzer

In [26]:
#df['clean_articles'] = df['articles']
from afinn import Afinn
afinn = Afinn(emoticons=True)
afinn_scores = [afinn.score(text) for text in df["clean_articles"]]
df['afinn_sentiment'] = afinn_scores

df[['clean_articles','articles','afinn_sentiment']]

,clean_articles,articles,afinn_sentiment
0,know covid vaccine booster oped nycs top doctor,What to Know About COVID-19 Vaccine Boosters: ...,2.0
1,choose covid vaccine booster shot,How to choose a COVID-19 vaccine booster shot,0.0
2,covid booster shot coming following federal ap...,COVID-19 booster shots coming following federa...,2.0
3,confused covid booster get guide help,Confused about which COVID booster to get? Thi...,0.0
4,least vaccinated american adult qualify covid ...,At least 89% of vaccinated American adults wil...,0.0
...,...,...,...
91,nearly million covid booster shot already admi...,Nearly 1 million Covid booster shots have alre...,0.0
92,covid booster shot coming here get,COVID-19 booster shots are coming; here's who ...,0.0
93,johnson johnson say booster shot provides incr...,Johnson & Johnson Says Its Booster Shot Provid...,1.0
94,moderna asked fda authorize booster covid vaccine,Moderna Has Asked The FDA To Authorize A Boost...,0.0


# Sentiment analysis using VADER 

(Valence Aware Dictionary and Sentiment Reasoner)

In [27]:
from nltk.sentiment import SentimentIntensityAnalyzer
#nltk.download('vader_lexicon')

nl_sent = SentimentIntensityAnalyzer()
nl_sent_score = [nl_sent.polarity_scores(text).get('compound') for text in df["clean_articles"]]
df['nltk_sentiment'] = nl_sent_score 

df[['clean_articles','articles','afinn_sentiment','nltk_sentiment']]

,clean_articles,articles,afinn_sentiment,nltk_sentiment
0,know covid vaccine booster oped nycs top doctor,What to Know About COVID-19 Vaccine Boosters: ...,2.0,0.2023
1,choose covid vaccine booster shot,How to choose a COVID-19 vaccine booster shot,0.0,0.0000
2,covid booster shot coming following federal ap...,COVID-19 booster shots coming following federa...,2.0,0.4767
3,confused covid booster get guide help,Confused about which COVID booster to get? Thi...,0.0,0.1027
4,least vaccinated american adult qualify covid ...,At least 89% of vaccinated American adults wil...,0.0,0.0000
...,...,...,...,...
91,nearly million covid booster shot already admi...,Nearly 1 million Covid booster shots have alre...,0.0,0.0000
92,covid booster shot coming here get,COVID-19 booster shots are coming; here's who ...,0.0,0.0000
93,johnson johnson say booster shot provides incr...,Johnson & Johnson Says Its Booster Shot Provid...,1.0,0.2732
94,moderna asked fda authorize booster covid vaccine,Moderna Has Asked The FDA To Authorize A Boost...,0.0,0.0000


In [28]:
# Select columns which has some sentiment score (either negative or positive)
df = (df.query("afinn_sentiment !=0 and nltk_sentiment  !=0")[0:20])
df

,articles,clean_articles,afinn_sentiment,nltk_sentiment
0,What to Know About COVID-19 Vaccine Boosters: ...,know covid vaccine booster oped nycs top doctor,2.0,0.2023
2,COVID-19 booster shots coming following federa...,covid booster shot coming following federal ap...,2.0,0.4767
6,Far fewer Californians getting COVID-19 booste...,far fewer californian getting covid booster sh...,-2.0,-0.3400
9,"No, COVID-19 vaccines or boosters will not cau...",covid vaccine booster cause test positive test,2.0,0.5574
21,F.D.A. to Allow ‘Mix and Match’ Approach for C...,fda allow mix match approach covid booster shot,1.0,0.2263
29,Covid-19 Booster Shots Are Available for All i...,covid booster shot available israel younger pe...,1.0,-0.3089
32,"To encourage COVID-19 booster shots, Philly of...",encourage covid booster shot philly official s...,2.0,0.5106
33,Want a COVID-19 booster shot? Here's what you ...,want covid booster shot here need know grand f...,7.0,0.7430
35,DH Deciphers | Covid-19 booster shots gain cur...,dh deciphers covid booster shot gain currency ...,2.0,0.5267
37,Sweden to extend COVID booster shots to all ag...,sweden extend covid booster shot aged,1.0,0.1779


# USING TRANSFORMERS PIPELINE SENTIMENT Analyzer

Based on DistilBert model

In [29]:
from transformers import pipeline
trans = pipeline('sentiment-analysis')

def get_score(article):
    db_sent = trans(article)
    db_score = db_sent[0].get('score')
    return db_score

def get_label (article):
    db_sent = trans(article)
    db_label = db_sent[0].get('label')
    return db_label

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
Some layers from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are 

In [30]:
db_sc = [get_score(article) for article in df["clean_articles"]]
df['DistilBert_score'] = db_sc
db_l = [get_label(article) for article in df["clean_articles"]]
df['DistilBert_label'] = db_l
df[['clean_articles','articles','afinn_sentiment','nltk_sentiment','DistilBert_score','DistilBert_label']]

,clean_articles,articles,afinn_sentiment,nltk_sentiment,DistilBert_score,DistilBert_label
0,know covid vaccine booster oped nycs top doctor,What to Know About COVID-19 Vaccine Boosters: ...,2.0,0.2023,0.958583,NEGATIVE
2,covid booster shot coming following federal ap...,COVID-19 booster shots coming following federa...,2.0,0.4767,0.867303,NEGATIVE
6,far fewer californian getting covid booster sh...,Far fewer Californians getting COVID-19 booste...,-2.0,-0.3400,0.988963,NEGATIVE
9,covid vaccine booster cause test positive test,"No, COVID-19 vaccines or boosters will not cau...",2.0,0.5574,0.967775,NEGATIVE
21,fda allow mix match approach covid booster shot,F.D.A. to Allow ‘Mix and Match’ Approach for C...,1.0,0.2263,0.983083,NEGATIVE
29,covid booster shot available israel younger pe...,Covid-19 Booster Shots Are Available for All i...,1.0,-0.3089,0.996219,NEGATIVE
32,encourage covid booster shot philly official s...,"To encourage COVID-19 booster shots, Philly of...",2.0,0.5106,0.628678,POSITIVE
33,want covid booster shot here need know grand f...,Want a COVID-19 booster shot? Here's what you ...,7.0,0.7430,0.998411,NEGATIVE
35,dh deciphers covid booster shot gain currency ...,DH Deciphers | Covid-19 booster shots gain cur...,2.0,0.5267,0.988345,POSITIVE
37,sweden extend covid booster shot aged,Sweden to extend COVID booster shots to all ag...,1.0,0.1779,0.964862,NEGATIVE


In [31]:
df.to_excel('titles_20.xlsx', index = False)